# Molecular dynamics from pretrained m3gnet model

In [1]:
import ase 
import warnings

from os                      import path
from ase.io                  import read, write
from ase.io.vasp             import write_vasp_xdatcar
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core           import Structure, Lattice
from m3gnet.models           import Relaxer, MolecularDynamics

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings('ignore', category=category, module='tensorflow')

2023-09-26 13:49:30.239258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 13:49:30.438095: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-26 13:49:31.209158: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-26 13:49:31.209811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
# Define basic parameters

compound = 'LiMg'
polymorf = 'Pm-3m'

temperature = 500  # In K
ensemble    = 'nvt'  # NVT canonical ensemble
timestep    = 1  # 1fs
steps       = 10000
loginterval = 10  # interval for record the log

# Define various paths

path_to_folder = f'Data/{compound}/{polymorf}'
path_to_POSCAR = f'{path_to_folder}/POSCAR'  # MP POSCAR (POSCAR_ini, DFT relaxed) is used

# If POSCAR is not there, we use the POSCAR_ini (DFT relaxed)
if not path.exists(path_to_POSCAR):
    system(f'cp {path_to_folder}/POSCAR_ini {path_to_folder}')

trajectory  = f'{path_to_folder}/md.traj'  # save trajectory to mo.traj
logfile     = f'{path_to_folder}/md.log'  # log file for MD

# Write simulation parameters into INCAR
with open(f'{path_to_folder}/INCAR', 'w') as file:
    file.write(f'TEBEG = {temperature}\n')
    file.write(f'POTIM = {timestep}\n')
    file.write(f'NBLOCK = {loginterval}')
    file.truncate()

In [3]:
# Load the MP structure
# DFT relaxed structure, supposed to work better

atoms = Structure.from_file(path_to_POSCAR)

In [4]:
# Run the simulation

md = MolecularDynamics(
    atoms=atoms,
    temperature=temperature,
    ensemble=ensemble,
    timestep=timestep, 
    trajectory=trajectory,
    logfile=logfile,
    loginterval=loginterval,
)

md.run(steps=steps)

2023-09-26 13:49:32.717635: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-26 13:49:32.717678: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (PC70744): /proc/driver/nvidia/version does not exist
2023-09-26 13:49:32.721521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/cibran.lopez/.local/lib/python3.10/site-packages/ase/md/nvtberendsen.py:95: RuntimeWarning: divide by zero encountered in double_scalars
  (self.temperature / old_temperature - 1.0) *


In [5]:
# Read the .traj file
atoms = read(trajectory, format='traj')

# Access the atomic structure information
cell      = atoms.get_cell()
positions = atoms.get_positions()
symbols   = atoms.get_chemical_symbols()

# Print the cell
print(cell)

# Print the atomic positions and symbols
for symbol, position in zip(symbols, positions):
    print(f'Atom symbol: {symbol}, Position: {position}')

Cell([17.076629999999998, 17.076629999999998, 17.076629999999998])
Atom symbol: Li, Position: [ 0.07843015  0.20594054 -0.18273237]
Atom symbol: Li, Position: [0.14816274 0.03931852 3.36460862]
Atom symbol: Li, Position: [0.12443895 0.18333604 7.08237212]
Atom symbol: Li, Position: [2.13857317e-01 7.81994376e-03 1.05985451e+01]
Atom symbol: Li, Position: [ 0.06418473  0.16181221 14.18181412]
Atom symbol: Li, Position: [-0.03136497  3.18911568 -0.4134566 ]
Atom symbol: Li, Position: [1.62260039e-03 3.55184820e+00 3.29483663e+00]
Atom symbol: Li, Position: [-0.16571752  3.25492482  7.10351567]
Atom symbol: Li, Position: [-0.3685687   3.38656617 10.90340206]
Atom symbol: Li, Position: [ 0.33012964  3.54198738 13.7407442 ]
Atom symbol: Li, Position: [-0.07801233  6.70241454 -0.38586583]
Atom symbol: Li, Position: [0.07915512 6.93088659 3.29611866]
Atom symbol: Li, Position: [-0.45483847  7.08791277  6.63779231]
Atom symbol: Li, Position: [-0.33162312  6.89987497 10.16652164]
Atom symbol: L

In [6]:
# Convert results to XDATCAR format
ase_traj = ase.io.read(trajectory, index=':')
ase.io.vasp.write_vasp_xdatcar(f'{path_to_folder}/XDATCAR', ase_traj, label=None)